In [1]:
import torch
from deepfake_detection.defaults import MODEL_OUT_PATH
from deepfake_detection.defaults import DATA_PATH, MODEL_OUT_PATH
import argparse
import os 
from deepfake_detection.training.configurator import get_config, TrainingConfig
from deepfake_detection.training.training_utils import Classifier, get_optimizer, get_scheduler
from torch.nn.modules.loss import BCEWithLogitsLoss
from deepfake_detection.training.data_loader import DeepFakeDetectionDataset
from torch.utils.data.dataloader import DataLoader
from deepfake_detection.training.augmentations import train_augmentations, val_augmentations
import json

/home/faraz/.cache/pypoetry/virtualenvs/deepfake-detection-luHkh4Hu-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:127: UserWarning: Field "model_key" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/faraz/.cache/pypoetry/virtualenvs/deepfake-detection-luHkh4Hu-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = MODEL_OUT_PATH / "big_batch_b4_25"
model_config_name = "default_config"

In [3]:
saved_model = torch.load(model_path)
saved_model['epoch']

25

In [4]:
from deepfake_detection.training.training_utils import Classifier
from deepfake_detection.training.configurator import get_config
config = get_config(config_name=model_config_name)
model = Classifier(model_key=config.model_key)

/home/faraz/Documents/code/deepfake_detection/deepfake_detection/training/training_utils.py:27: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  self.model = config['init_func'](**config['init_args'])


In [5]:
model.load_state_dict(saved_model['state'])

<All keys matched successfully>

In [6]:
from deepfake_detection.training.data_loader import DeepFakeDetectionDataset

data_val = DeepFakeDetectionDataset(
    mode="val",
    data_path="val.csv",
    data_folder_path=DATA_PATH,
    augmentations=val_augmentations(height=config.img_height, width=config.img_width)
)    
val_loader = DataLoader(
    data_val, 
    batch_size=config.batch_size, 
    pin_memory=False,
    shuffle=True,
    drop_last=True
)

In [25]:
import cv2
import numpy as np 
model = model.eval()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
with torch.no_grad():
    for data in val_loader:
        images = data[0]
        labels = data[1]
    
        images = images.to(device)
        labels = labels.to(device)

        out_labels = model(images)
        
        out_labels[out_labels < 0] = 0
        out_labels[out_labels > 0] = 1

        ground_truth = labels.detach().cpu().numpy()
        preds = out_labels.detach().cpu().numpy().flatten()

        print(f"Ground Truth: {ground_truth}")
        print(f"Predictions: {preds}")
        percentage_match = sum(ground_truth == preds) / len(ground_truth)
        print(f"Percentage Match: {percentage_match * 100}%")
        break

Ground Truth: [1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
Predictions: [1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
Percentage Match: 95.0%
